# JIT Engine: Scalar x Tensor

This example will go over how to compile MLIR code for multiplying a scalar by a tensor. 

Previous tutorials have gone over how to broadcast vectors. For the simple task of multiplying a each tensor's elements by a scalar, broadcasting may be unwarranted or unnecessary. We'll go over how to implement this in a much simpler and more straightforward fashion. 

Let’s first import some necessary modules and generate an instance of our JIT engine.

In [1]:
import mlir_graphblas
import numpy as np

engine = mlir_graphblas.MlirJitEngine()

Here's the MLIR code we'll use. 

In [2]:
mlir_text = """
#trait_add = {
 indexing_maps = [
   affine_map<(i, j) -> (i, j)>,
   affine_map<(i, j) -> (i, j)>
 ],
 iterator_types = ["parallel", "parallel"]
}

func @scale(%arg_tensor: tensor<2x3xf32>, %arg_scalar: f32) -> tensor<2x3xf32> {
  %output_storage = constant dense<0.0> : tensor<2x3xf32>
  %answer = linalg.generic #trait_add
    ins(%arg_tensor: tensor<2x3xf32>)
    outs(%arg_tensor: tensor<2x3xf32>) {
      ^bb(%a: f32, %s: f32):
        %scaled = mulf %a, %arg_scalar : f32
        linalg.yield %scaled : f32
    } -> tensor<2x3xf32>
 return %answer : tensor<2x3xf32>
}
"""

These are the passes we'll utilize. 

In [3]:
passes = [
    "--linalg-bufferize",
    "--func-bufferize",
    "--tensor-bufferize",
    "--tensor-constant-bufferize",
    "--convert-linalg-to-loops",
    "--finalizing-bufferize",
    "--convert-scf-to-std",
    "--convert-std-to-llvm",
]

Let's compile our MLIR code. 

In [4]:
engine.add(mlir_text, passes)

['scale']

Let's try out our compiled function. 

In [5]:
# grab our callable
scale = engine.scale

# generate inputs
a = np.arange(6, dtype=np.float32).reshape([2,3])

# generate output
result = scale(a, 100)

In [6]:
result

array([[  0., 100., 200.],
       [300., 400., 500.]], dtype=float32)

Let's verify that our function works as expected.

In [8]:
np.all(result == a*100)

True